# Lecture 12:  AB Testing
### Data Science 1: CS S109A <br>  Instructors: P. Protopapas, K. Rader
#### Harvard University <br> Summer 2018 <br> 

---

Import *way too many* libraries

In [26]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
sns.set(style="ticks")
%matplotlib inline

%matplotlib inline

### Read in the data

In [3]:
# Load dataset_1
data_bone = pd.read_csv('bone.csv', delimiter=',')
data_bone.head()

rat    group  treatment  bone density  lowjump  highjump
0    1  Control          1           611        0         0
1    2  Control          1           621        0         0
2    3  Control          1           614        0         0
3    4  Control          1           593        0         0
4    5  Control          1           593        0         0

In [4]:
#creating the bone density measures for each group
controlgroup = data_bone['bone density'][data_bone['group']=='Control']
trt1group = data_bone['bone density'][data_bone['group']=='Lowjump']
trt2group = data_bone['bone density'][data_bone['group']=='Highjump']

#two sample t-test
sp.stats.ttest_ind(controlgroup,trt1group)

Ttest_indResult(statistic=-1.0760571105207466, pvalue=0.29611370541618665)

In [5]:
#ANOVA F-test
sp.stats.f_oneway(controlgroup,trt1group,trt2group)

F_onewayResult(statistic=7.977836956953773, pvalue=0.0018951062682877964)

In [9]:
#creating indicator variables for each group
ybar = np.mean(data_bone['bone density'])

controlgroup_ind = 1*(data_bone['bone density'][data_bone['group']=='Control']>ybar)
trt1group_ind = 1*(data_bone['bone density'][data_bone['group']=='Lowjump']>ybar)
trt2group_ind = 1*(data_bone['bone density'][data_bone['group']=='Highjump']>ybar)

#let's see if the indicator is different across Control vs. Other
mytable1 = pd.crosstab(data_bone['group']=='Control',data_bone['bone density']>ybar)
mytable1

bone density  False  True 
group                     
False             6     14
True              8      2

In [22]:
mytable1.iloc[:,1]

group
False    14
True      2
Name: True, dtype: int64

In [27]:
# z-test for proportions
xs = mytable1.iloc[:,1]
ns = mytable1.sum(axis=1)
proportions_ztest(xs,ns)


(2.587745847533828, 0.0096606229974849581)

In [11]:
#Fisher Exact test
mytable = pd.crosstab(data_bone['group']=='Control',data_bone['bone density']>ybar)

sp.stats.fisher_exact(mytable1)



(0.10714285714285714, 0.018690654672663655)

In [28]:
mytable2 = pd.crosstab(data_bone['group'],data_bone['bone density']>ybar)
sp.stats.chi2_contingency(mytable2)

(13.928571428571431,
 0.00094503772154749571,
 2,
 array([[ 4.66666667,  5.33333333],
        [ 4.66666667,  5.33333333],
        [ 4.66666667,  5.33333333]]))

In [23]:
mytable2

bone density  False  True 
group                     
Control           8      2
Highjump          0     10
Lowjump           6      4